# Grouping and aggregating data #
This guy has a great tutorial on <a href="https://www.shanelynn.ie/summarising-aggregation-and-grouping-data-in-python-pandas/" target="_blank">Summarising, Aggregating, and Grouping data in Pandas.</a>
### Motivating questions ###
We need to group data and calculate aggregate values to answer questions such as:
1. How do teams compare to each other? - Group rows of data by team.
2. Can we compare a player's strikeouts from one year to the next? - Group rows of data by year.
3. Can we compare a team's home and road performance, or performance at different parks? - Group rows of data by game location.

### SQL vs. Pandas ###
If you're familiar with SQL, you might find it helpful to see how the SQL commands compare to the equivalent Pandas commands.
<a href="https://pandas.pydata.org/pandas-docs/stable/getting_started/comparison/comparison_with_sql.html" target="_blank">Pandas and SQL</a>

### Pandas groupby###
There are times when you want to group the individual rows in a dataframe by some parameter, perform a calculation on the group, and then recombine the results. Use GROUP BY to split the data into groups, then aggregate the data in each group, then display the aggregate calculations as one or more columns in a dataframe or series.

**Example:**
How many pitchers did each MLB team use in 2015?
1. Read the pitching tables, which contains each individual pitcher that appeared in 2015.

In [88]:
#import pandas as pd
#from pybaseball.lahman import *

#look in pitching
#pitch = pitching()
#pitch2015 = pitch.loc[(pitch['yearID']==2015)].copy()
#pitch2015.head()

2. Group by teamID.
3. Return the size of the group, or the count of a field in the group.

In [89]:
#p = pitch2015.groupby('teamID').size()
#print(p)
##or
#p2 = pitch2015.groupby('teamID')['playerID'].count()
#print(p2)
#How would we verify that our groupby worked?

### Built-in aggregations ###
* count() - number of items
* first(), last() - first and last item
* mean(), median() - mean and median
* min(), max() - minimum and maximum values
* std(), var() - standard deviation and variance
* mad() - mean absolute deviation
* prod() - product of all items
* sum() - sum of all items

**Question:** How many hits did each pitching staff give up in 2015? The abbreviation for Hits is H.

**Example:** How many hits, walks, and homeruns did each team give up in 2015?
We can return the sum of each column in our group.

In [90]:
#pHits = pitch2015.groupby('teamID')['H','BB','HR'].sum()
#print(pHits)

## Multiple statistics on the group ##
With groupby, we can calculate one statistic on multiple columns. But, what if we need multiple statistics on a column, or multiple statistics on multiple columns. You might also need to create a custom statistic.

**Example:** What was the mean ERA, number of hits, homeruns, walks, and pitchers used for each pitching staff in 2015? 
To answer this, we need the mean of the ERA column and the sum of the other columns.
We can apply the aggregate function to our groups and list how we want the data aggregated. 

To start, we can calculate the count of pitchers the same as we did above using agg. We get the same result.

In [91]:
#p = pitch2015.groupby(['teamID']).agg({'playerID':'count'})
#print(p)

**Example:** What was each team's count of pitchers and team ERA?
We need multiple calculations this time.

In [92]:
#p = pitch2015.groupby(['teamID']).agg(
#    {'playerID':'count','ERA':'mean'}
#    )
#print(p)

**Example:** What was the mean, min, and max ERA for each team and number of pitchers used?

In [93]:
#p = pitch2015.groupby(['teamID']).agg(
#    {'playerID':'count','ERA':['mean', 'min','max']}
#    )
#print(p)

**Question:** Add the total hits, homeruns, and walks to the previous groupby.

## Grouping by multiple columns ##
In our groupby example, we only used one column - teamID. But, what if we wanted to know how teams performed over time? We would need to group by team and year. We'll limit to the last 10 years just so that we don't have so much data.

In [94]:
#pitchAllYears = pitch.loc[(pitch['yearID']> 2005)].copy()
#pitchYearsGrouped = pitchAllYears.groupby(['yearID','teamID']).size()
#print(pitchYearsGrouped)

## What is an index? ##
All dataframes have a column that serves as the name of the row. This is called an *index*. By default, the index is the row number, and we can see that when we create a dataframe and print out a few rows and the index.

In [95]:
#print(pitch.head())
##index of dataframe
#print(pitch.index)

### An index is unique ###
The row number is printed as the first column and when we access data by its row index, that's the value that we're referring to, in this case. The index is a unique value. Two rows can't have the same index. You can set the index on a dataframe, and you can check if your selected column has duplicate values and generate an error if it does.

In [96]:
#pIndex = pitch.copy()
#pIndex.set_index('playerID', drop=False, verify_integrity=True) #This will raise an error

### Groupby changes the index ###
When you call groupby on a dataframe, you get the group criteria as the new index, unless you specify otherwise. Depending on what you're trying to do, you might not want this.

In [97]:
#pitching data from previous query. Notice that the rows aren't numbered
#print(p)
#print(p.index)

### Multi-index dataframes ###
If you're groupby has multiple columns, your index will be the combination of those columns. This is called a multi-index, or hierarchical index. We see this if we look at the *pitchAllYears* dataframe. We access the data using the index.

In [98]:
#print(pitchYearsGrouped.index)
#print(pitchYearsGrouped.loc[(2006,'COL')])

### Preserving the index ###
One approach to indexing is to keep the original index column and just let groupby reindex the rows. You do this with as_index=False.

In [99]:
#pitchAllYears2 = pitch.loc[(pitch['yearID']> 2005)].copy()
#pitchYearsGrouped2 = pitchAllYears2.groupby(['yearID','teamID'], as_index=False).agg({"playerID": "count"})
#print(pitchYearsGrouped2.index)
#print(pitchYearsGrouped2)

## An indexing example ##
Add the name of the manager for each team in the (team, year) dataframe just generated. One approach is to read managers and merge on teamID and yearID to generate a new dataframe. The other approach is to generate a (teamID, yearID) multi-index on the dataframe and merge on the index. Do both approaches work?

In [100]:
#man = managers()
#man = man.loc[(man['yearID'] > 2005)]
#print(man)

In [101]:
#teamsAndManagers = pd.merge(pitchYearsGrouped2, man[['teamID','yearID','playerID']], on=['yearID', 'teamID'], how='inner')
#print(teamsAndManagers)
##you can mess with the column labels to clean it up and do an additional merge to get the managers name from master

In [102]:
##Merging using the index
##What happens?
#pitchYearsGrouped2 = pitchAllYears2.groupby(['yearID','teamID']).agg({"playerID": "count"})

#man2 = managers()
#man2 = man2.loc[(man2['yearID'] > 2005)]
#man2.set_index(['teamID', 'yearID'],drop=False, verify_integrity=True)


## Questions: ##
In addition to the couple of questions previously mentioned, consider the following:

Imagine you work for the Colorado Rockies and you've been tasked with improving the team's pitching staff. The Rockies are considering hiring a manager who used to be a Major League pitcher. Their argument is, since he used to be a pitcher, he'll know how to build a good pitching staff.

Is there any evidence in the data that teams managed by former Major League pitchers have better pitching staffs than other teams?
Before diving into the data, describe how you will answer this question. 
1. What data do you need?
2. What argument will you make?
3. What are the limitations to your argument?
4. If time permits, you can start working on answering this question.
